# VLA Navigation - Live Demo

This notebook demonstrates live VLA-guided navigation on JetBot.

## Prerequisites

1. VLA server running on a GPU machine:
   ```bash
   cd server/vla_server
   python server.py --port 5555
   ```

2. Network connectivity between JetBot and GPU server

## How It Works

1. JetBot captures camera frames at 224x224
2. Frames are sent to the VLA server with a natural language instruction
3. OpenVLA model predicts motor commands
4. JetBot executes the commands to navigate

## Safety Features

- Speed limiting (default: 50% max)
- Connection timeout auto-stop
- Emergency stop button

### Import Libraries

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

from jetbot import Robot, Camera, bgr8_to_jpeg
from jetbot.vla import VLAClient

import time
import threading

### Configuration

In [ ]:
# VLA Server Configuration
SERVER_HOST = 'localhost'  # Change to your GPU server IP
SERVER_PORT = 5555

# Safety Configuration
MAX_SPEED = 0.5  # Maximum motor speed (0.0 to 1.0)
CONNECTION_TIMEOUT = 1.0  # Seconds before stopping on connection loss

### Initialize Components

In [ ]:
# Initialize camera
camera = Camera.instance(width=224, height=224)
time.sleep(1)

# Initialize robot
robot = Robot()

print('Camera and robot initialized')

### Connect to VLA Server

In [ ]:
vla_client = VLAClient(
    server_host=SERVER_HOST,
    server_port=SERVER_PORT,
    timeout_ms=500
)

if vla_client.connected:
    print(f'Connected to VLA server at {SERVER_HOST}:{SERVER_PORT}')
else:
    print(f'Warning: Could not connect to VLA server at {SERVER_HOST}:{SERVER_PORT}')
    print('Make sure the server is running!')

### Create UI Widgets

In [ ]:
# Camera display
image_widget = widgets.Image(format='jpeg', width=224, height=224)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# Instruction input
instruction_widget = widgets.Text(
    value='navigate forward avoiding obstacles',
    placeholder='Enter instruction...',
    description='Instruction:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Quick instruction buttons
quick_instructions = [
    'navigate forward avoiding obstacles',
    'go to the left',
    'go to the right',
    'turn around',
    'stop and wait',
    'explore the room'
]

def set_instruction(instruction):
    def handler(b):
        instruction_widget.value = instruction
        vla_client.instruction = instruction
    return handler

quick_buttons = []
for instr in quick_instructions:
    btn = widgets.Button(
        description=instr[:20] + '...' if len(instr) > 20 else instr,
        layout=widgets.Layout(width='auto')
    )
    btn.on_click(set_instruction(instr))
    quick_buttons.append(btn)

# Speed slider
speed_slider = widgets.FloatSlider(
    value=MAX_SPEED,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Max Speed:',
    style={'description_width': 'initial'}
)

# Status displays
connection_widget = widgets.HTML(
    value='<span style="color: gray;">Disconnected</span>'
)

latency_widget = widgets.Label(value='Latency: --ms')

left_speed_widget = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1,
    description='Left:',
    bar_style='info'
)

right_speed_widget = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1,
    description='Right:',
    bar_style='info'
)

# Control buttons
start_button = widgets.Button(
    description='Start',
    button_style='success',
    icon='play'
)

stop_button = widgets.Button(
    description='STOP',
    button_style='danger',
    icon='stop'
)

### VLA Control Loop

In [ ]:
# Control state
running = False
last_command_time = time.time()

def update_callback(change):
    global running, last_command_time
    
    if not running:
        return
    
    image = change['new']
    
    # Update instruction
    vla_client.instruction = instruction_widget.value
    
    # Get VLA prediction
    left_speed, right_speed = vla_client.predict(image)
    
    # Update connection status
    if vla_client.connected:
        connection_widget.value = '<span style="color: green;">Connected</span>'
        latency_widget.value = f'Latency: {vla_client.latency_ms:.0f}ms'
        last_command_time = time.time()
    else:
        connection_widget.value = '<span style="color: red;">Disconnected</span>'
        
        # Safety: stop if connection lost too long
        if time.time() - last_command_time > CONNECTION_TIMEOUT:
            robot.stop()
            return
    
    # Apply speed limit
    max_speed = speed_slider.value
    left_speed = max(-max_speed, min(max_speed, left_speed))
    right_speed = max(-max_speed, min(max_speed, right_speed))
    
    # Update motor displays (normalize to 0-1 for progress bar)
    left_speed_widget.value = (left_speed + 1) / 2
    right_speed_widget.value = (right_speed + 1) / 2
    
    # Update bar style based on direction
    left_speed_widget.bar_style = 'success' if left_speed >= 0 else 'danger'
    right_speed_widget.bar_style = 'success' if right_speed >= 0 else 'danger'
    
    # Set motor speeds
    robot.set_motors(left_speed, right_speed)

def start_navigation(b):
    global running
    running = True
    camera.observe(update_callback, names='value')
    start_button.disabled = True
    stop_button.disabled = False
    print(f'Started with instruction: "{instruction_widget.value}"')

def stop_navigation(b):
    global running
    running = False
    camera.unobserve(update_callback, names='value')
    robot.stop()
    start_button.disabled = False
    stop_button.disabled = True
    print('Stopped')

start_button.on_click(start_navigation)
stop_button.on_click(stop_navigation)
stop_button.disabled = True

### Display Interface

In [ ]:
display(widgets.VBox([
    widgets.HBox([image_widget]),
    instruction_widget,
    widgets.HBox(quick_buttons[:3]),
    widgets.HBox(quick_buttons[3:]),
    speed_slider,
    widgets.HBox([connection_widget, latency_widget]),
    left_speed_widget,
    right_speed_widget,
    widgets.HBox([start_button, stop_button])
]))

### Cleanup

In [ ]:
# Run this cell when done to clean up
running = False
robot.stop()
camera.stop()
vla_client.close()
print('Cleanup complete')